In [12]:
# this file generates Unity/C# code by extracting all available
# colormaps from matplotlib to use them as Gradients in Unity

import matplotlib.pyplot as plt
import numpy as np
import time

def generate_csharp_gradient_color_keys(colormap_name):

  # Create a colormap object
  cmap = plt.get_cmap(colormap_name)

  # Get the number of values in the colormap
  num_values = cmap.N

  # Sample the colormap
  values = np.linspace(0, 1, num_values)

  # Initialize an empty list to store C# code
  csharp_code = []

  # Generate C# code for each sampled color
  for value in values:
      rgba = cmap(value)
      r, g, b, a = rgba
      time = format(value, '.7f')
      csharp_code.append(f'''            new GradientColorKey(new Color({r}f, {g}f, {b}f, {a}f), {time}f)''')

  # Join the code lines into a single string
  csharp_array = ',\n'.join(csharp_code)

  # Create the final C# code string
  csharp_string = f'''        public static GradientColorKey[] {colormap_name}Array = new GradientColorKey[] {{\n{csharp_array}\n        }};'''

  return csharp_string


#colormap_name = 'viridis'
#csharp_code = generate_csharp_gradient_color_keys(colormap_name)
#print(csharp_code)

# Get a list of all known colormaps
all_colormaps = plt.colormaps()

# Define the output file path
output_file1 = 'ColormapArrays.cs'
output_file2 = 'ColormapProperties.cs'
output_file3 = 'Colormaps.cs'

header = '''//------------------------------------------------------------------------------
// <auto-generated>
//     This code was generated by a tool.
// </auto-generated>
//------------------------------------------------------------------------------

using UnityEngine;

namespace Chromatic {

    public static partial class ColorMaps {

'''

footer = '''    }
}'''

# Generate C# code for values of each colormap and write it to static arrays
with open(output_file1, 'w') as file:

    file.write(header)

    for colormap_name in all_colormaps:
        csharp_code = generate_csharp_gradient_color_keys(colormap_name)
        file.write(csharp_code + '\n')

    file.write(footer)

print(f'C# code has been written to {output_file1}')


# Generate C# properties for each colormap
with open(output_file2, 'w') as file:

    file.write(header)

    for colormap_name in all_colormaps:
        # Create C# property code with the current colormap name
        property_code = f'''
        public static Gradient {colormap_name} {{
            get {{
                if (_{colormap_name} == null) {{
                    _{colormap_name} = new Gradient();
                    _{colormap_name}.SetKeys({colormap_name}Array, null);
                }}
                return _{colormap_name};
            }}
        }}
        private static Gradient _{colormap_name};
        '''

        file.write(property_code)

    file.write(footer)

print(f'C# code has been written to {output_file2}')


# Generate a C# list that contains all colormaps
with open(output_file3, 'w') as file:

    file.write(header)

    maps = []

    for colormap_name in all_colormaps:
        maps.append(f'''            {colormap_name}''')

    maps_array = ',\n'.join(maps)

    file.write(f'''        public static Gradient[] Colormaps = new Gradient[] {{\n{maps_array}\n        }};\n''')

    file.write(footer)

print(f'C# code has been written to {output_file3}')


C# code has been written to ColormapArrays.cs
C# code has been written to ColormapProperties.cs
C# code has been written to Colormaps.cs
